In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
from sklearn.datasets import fetch_20newsgroups

def load_news(fname=None, all_categories=False):
    ''' fname is useless but its there to follow the interface convention'''
    # Load some categories from the training set
    if all_categories:
        categories = None
    else:
        categories = [
            'alt.atheism',
            'talk.religion.misc',
            'comp.graphics',
            'sci.space',
            ]

    print("Loading 20 newsgroups dataset for categories:")
    print(categories if categories else "all")

    data_train = fetch_20newsgroups(subset='train', categories=categories,
                                    shuffle=True, random_state=42)

    data_test = fetch_20newsgroups(subset='test', categories=categories,
                                   shuffle=True, random_state=42)
    print('data loaded')

    categories = data_train.target_names    # for case categories == None

    print("%d categories" % len(categories))
    print()

    # split a training set and a test set
    return data_train, data_test

In [3]:
# example of clustering visualisation using scikit-learn

METHODS = 'random,pca,lda,isomap,lle,mlle,hlle,ltsa,mds,rtree,spectral,tsne'

import numpy as np
import traceback
from time import time
from random import choice, random
from sklearn import manifold, decomposition, ensemble, lda, random_projection

from mlboost.core.pphisto import SortHistogram, Histogram


def plot_embedding(X, Y, title=None, n_sample_by_class=2, min_freq=100,
                   sampling=.1, legend_outside_box=True, enable_legend_picking=False, source=None):
    '''Scale and visualize the embedding vectors'''

    # Make sure there is no line with NaN value
    nans = np.isnan(X)
    retained = np.invert(np.any(nans, 1))
    retained_idx = np.where(retained)[0]
    X = X[retained_idx, :]
    if len(X) == 0:
        # Nothing to plot
        return False

    import pylab as pl
    y = np.array(Y,dtype=str)
    x_min, x_max = np.min(X, 0), np.max(X, 0)
    X = (X - x_min) / (x_max - x_min)

    def get_point_info(x,y):
        for i, row in enumerate(X):
            if row[0]==x and row[1]==y:
                msg = "[{x},{y}] = row {i}".format(x=x,y=y,i=i)
                if source:
                    msg+='-> class : {c}\n{content}'.format(c=Y[i],content=source[i])
                print msg

    fig = pl.figure()
    ax = pl.subplot(111)

    lines = [] # this is used to enable picking on legend (see below)

    # show untagged data points
    tidx = np.where(y=='?')[0]
    if len(tidx):
        sX = X[tidx,:]
        sX = sX[[i for i in range(len(tidx)) if random()<sampling],:]
        if len(sX):
            lines.append(ax.plot(sX[:,0],sX[:,1],'.', label='?', picker=True)[0])

    # show some tagged samples
    tidx = np.where(y!='?')[0]

    dist = Histogram(y[tidx])
    sdist = SortHistogram(dist, False, True)
    classes = [label for label, count in sdist if count>min_freq]
    classes.sort()

    for cl in classes:
        tidx = np.where(y==str(cl))[0]
        cX = X[tidx,:]
        color = float(classes.index(cl))/len(classes)
        lines.append(ax.plot(cX[:,0],cX[:,1],'.', color=pl.cm.Set1(color), label=cl, picker=True)[0])

        # show some label on the graph
        for i in range(n_sample_by_class):
            if len(cX)>0:
                idx = choice(range(len(cX)))
                pl.text(cX[idx, 0], cX[idx, 1], str(cl),
                        color=pl.cm.Set1(float(classes.index(cl))/len(classes)),
                        fontdict={'weight': 'bold', 'size': 9})


    if legend_outside_box:
        # Shink current axis by 20%
        box = ax.get_position()
        ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])

        # Put a legend to the right of the current axis
        leg = ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    else:
        leg = pl.legend()

    if enable_legend_picking:
        # enable picking on the legend
        lined = dict()

        for legline, origline in zip(leg.get_lines(), lines):
            legline.set_picker(5)  # 5 pts tolerance
            lined[legline] = origline

        def onpick(event):
            # on the pick event, find the orig line corresponding to the
            # legend proxy line, and toggle the visibility
            legline = event.artist
            origline = lined[legline]
            vis = not origline.get_visible()
            origline.set_visible(vis)
            # Change the alpha on the line in the legend so we can see what lines
            # have been toggled
            if vis:
                legline.set_alpha(1.0)
            else:
                legline.set_alpha(0.2)
            fig.canvas.draw()

        fig.canvas.mpl_connect('pick_event', onpick)
    else:
        # point picking (with data)
        from matplotlib.lines import Line2D
        def onpick1(event):
            if isinstance(event.artist, Line2D):
                thisline = event.artist
                xdata = thisline.get_xdata()
                ydata = thisline.get_ydata()
                ind = event.ind
                x=np.take(xdata, ind)
                y=np.take(ydata, ind)
                n=len(x)+1
                xy = zip(x,y)
                print('%i points: ' %n)
                for i,(x,y) in enumerate(xy):
                    print("#%i)" %(i+1))
                    get_point_info(x, y)

        fig.canvas.mpl_connect('pick_event', onpick1)

    if title is not None:
        pl.title(title)

    pl.xlabel('dim 1')
    pl.ylabel('dim 2')
    return True

def randomp(X, dim=2, **kargs):
    '''Random 2D projection using a random unitary matrix'''
    print("Computing random projection")
    try:
        rp = random_projection.SparseRandomProjection(n_components=dim, random_state=42)
        X_projected = rp.fit_transform(X)
        return rp, X_projected, "Random Projection"
    except Exception as e:
        traceback.print_exc()

def pca(X, dim=2, **kargs):
    '''Projection on to the first 2 principal components'''

    print("Computing PCA projection")
    try:
        pca = decomposition.RandomizedPCA(n_components=dim)
        X_pca = pca.fit_transform(X)
        return pca, X_pca, "Principal Components projection"
    except Exception as e:
        traceback.print_exc()

class ldaModel(object):
    def __init__(self, model):
        self.model = model

    def getInvertible(self, X):
        X2 = X.copy()
        X2.flat[::X.shape[1] + 1] += 0.01  # Make X invertible
        return X2

    def fit(self, X, y=None):
        X2 = self.getInvertible(X)
        return self.model.fit(X2, y)

    def fit_transform(self, X, y=None):
        X2 = self.getInvertible(X)
        return self.model.fit_transform(X2, y)

    def transform(self, X):
        X2 = self.getInvertible(X)
        return self.model.transform(X2)

    def predict(self, X):
        X2 = self.getInvertible(X)
        return self.model.predict(X2)

def lda(X, Y, dim=2, **kargs):
    '''Projection on to the first 2 linear discriminant components'''
    from sklearn import lda
    print("Computing LDA projection")
    try:
        ldatr = ldaModel(lda.LDA(n_components=dim))
        X_lda = ldatr.fit_transform(X, Y)
        return ldatr, X_lda, "Linear Discriminant projection of the features"
    except Exception as e:
        traceback.print_exc()

def isomap(X, dim=2, n_neighbors=30, **kargs):
    '''Isomap projection of the dataset'''
    print("Computing Isomap embedding")
    try:
        isomap = manifold.Isomap(n_neighbors, n_components=dim)
        X_iso = isomap.fit_transform(X)
        print("Done.")
        return isomap, X_iso, "Isomap projection of the features"
    except Exception as e:
        traceback.print_exc()

def lle(X, dim=2, n_neighbors=30, **kargs):
    '''Locally linear embedding of the dataset'''
    print("Computing LLE embedding")
    methods = ['standard', 'ltsa', 'hessian', 'modified']
    clf = manifold.LocallyLinearEmbedding(n_neighbors, n_components=dim,
                                          eigen_solver='auto',
                                          method='standard')
    try:
        X_lle = clf.fit_transform(X)
        print("Done. Reconstruction error: %g" % clf.reconstruction_error_)
        return clf, X_lle, "Locally Linear Embedding of the features"

    except Exception as e:
        traceback.print_exc()

def mlle(X, dim=2, n_neighbors=30, **kargs):
    '''Modified Locally linear embedding of the dataset'''
    print("Computing modified LLE embedding")
    clf = manifold.LocallyLinearEmbedding(n_neighbors, n_components=dim,
                                          method='modified')
    try:
        X_mlle = clf.fit_transform(X)
        print("Done. Reconstruction error: %g" % clf.reconstruction_error_)
        return clf, X_mlle, "Modified Locally Linear Embedding of the features"
    except Exception as e:
        traceback.print_exc()

def hlle(X, dim=2, n_neighbors=30, **kargs):
    '''HLLE embedding of the dataset'''
    print("Computing Hessian LLE embedding")
    clf = manifold.LocallyLinearEmbedding(n_neighbors, n_components=dim,
                                          method='hessian')
    try:
        X_hlle = clf.fit_transform(X)
        print("Done. Reconstruction error: %g" % clf.reconstruction_error_)
        return clf, X_hlle, "Hessian Locally Linear Embedding of the features"
    except Exception as e:
        traceback.print_exc()

def ltsa(X, dim=2, n_neighbors=30, **kargs):
    '''LTSA embedding of the dataset'''
    print("Computing LTSA embedding")
    clf = manifold.LocallyLinearEmbedding(n_neighbors, n_components=dim,
                                          method='ltsa')
    try:
        X_ltsa = clf.fit_transform(X)
        print("Done. Reconstruction error: %g" % clf.reconstruction_error_)
        return clf, X_ltsa, "Local Tangent Space Alignment of the features"

    except Exception as e:
        traceback.print_exc()

def mds(X, dim=2, **kargs):
    '''MDS  embedding of the dataset'''
    print("Computing MDS embedding")
    clf = manifold.MDS(n_components=dim, n_init=1, max_iter=100)

    try:
        X_mds = clf.fit_transform(X)
        print("Done. Stress: %f" % clf.stress_)
        return clf, X_mds, "MDS embedding of the features"
    except Exception as e:
        traceback.print_exc()

def rtree(X, dim=2, n_estimators=200, max_depth=5, **kargs):
    '''Random Trees embedding of the dataset'''
    print("Computing Totally Random Trees embedding")
    from sklearn.pipeline import Pipeline
    tr = Pipeline([
        ('hasher', ensemble.RandomTreesEmbedding(n_estimators=n_estimators, random_state=0,
                                           max_depth=max_depth)),
        ('pca', decomposition.RandomizedPCA(n_components=dim))])

    try:
        X_reduced = tr.fit_transform(X)

        return tr, X_reduced, "Random forest embedding of the features"
    except Exception as e:
        traceback.print_exc()


def spectral(X, dim=2, **kargs):
    # Spectral embedding of the dataset
    # quite cool https://github.com/oreillymedia/t-SNE-tutorial
    print("Computing Spectral embedding")
    embedder = manifold.SpectralEmbedding(n_components=dim, random_state=0,
                                          eigen_solver="arpack")
    try:
        X_se = embedder.fit_transform(X)

        return embedder, X_se, "Spectral embedding of the features"

    except Exception as e:
        traceback.print_exc()

def tsne(X, dim=2, **kargs):
    # t-sne embedding of the dataset
    print("Computing t-sne embedding")
    embedder = manifold.TSNE(n_components=dim, init='pca', random_state=0)
    try:
        X_se = embedder.fit_transform(X)

        return embedder, X_se, "t-sne embedding of the features"

    except Exception as e:
        traceback.print_exc()

_fct_map = {
    "random":randomp,
    "pca":pca,
    "lda":lda,
    "isomap":isomap,
    "lle":lle,
    "mlle":mlle,
    "hlle":hlle,
    "ltsa":ltsa,
    'tsne':tsne,
    # MDS is lacking a transform method, so cannot be used
    # to fit a transform on training set and project on the test set.
#    "mds":mds,
    "rtree":rtree,
    # SpectralEmbedding is lacking a transform method, so cannot be used
    # to fit a transform on training set, and project on the test set.
#    "spectral":spectral
    }

def dim_reduce(algo_name, **kwargs):
    if algo_name not in _fct_map:
        print "warning: %s not available" %algo_name
    else:
        return _fct_map[algo_name](**kwargs)

In [ ]:
# example of clustering visualisation using scikit-learn

import sys
import traceback
import matplotlib
import warnings
# load interface; return data_train, data_test that contains data & targets
#from news import load_news

def filter_classes(X, Y, classes, remove):
    ''' remove classes points from X and Y '''
    print "FILTERING CLASSES"
    import numpy as np
    cidx=[]
    for c in np.array(classes,dtype=int):
        cidx.extend(np.where(Y==c)[0])
    if remove:
        idx=[i for i in range(len(Y)) if i not in cidx]
    else:
        idx=[i for i in range(len(Y)) if i in cidx]
    idx = np.array(idx)
    print 'selecting %i/%i indexes' %(len(idx),len(X))
    return X[idx], Y[idx]

_load_map = {'news':load_news}

def add_loading_dataset_fct(name, fct):
    if name in _load_map:
        print "load name already used %s" %name
        sys.exit(1)
    else:
        _load_map[name] = fct

def size_mb(docs):
    try:
        return sum(len(s.encode('utf-8')) for s in docs) / 1e6
    #return sum(len(s) for s in docs) / 1e6
    except:
        return 0

def main(args=None):
    args = args or sys.argv[1:]
    import logging
    import numpy as np
    from optparse import OptionParser
    from time import time
    from sklearn.feature_extraction.text import TfidfVectorizer, HashingVectorizer
    from sklearn.feature_selection import SelectKBest, chi2
    #import dim_reduction as dr

    # Display progress logs on stdout
    logging.basicConfig(level=logging.INFO,
                        format='%(asctime)s %(levelname)s %(message)s')

    # parse commandline arguments
    op = OptionParser()

    op.add_option("--chi2_select", default=-1,
                  action="store", type="int", dest="select_chi2",
                  help="Select some number of features using a chi-squared test; all set -1")
    op.add_option('-f',"--filename", default="data.tsv",
		  dest="fname", help="data filename")
    op.add_option("-d", "--dataset", default='news',
                  dest="dataset",
                  help="dataset to load (%s)" %_load_map.keys())
    op.add_option("--n_features",
                  action="store", type=int, default=1000,
                  help="n_features when using the hashing vectorizer.")
    op.add_option("--use_hashing", default=False,
                  action="store_true",
                  help="Use a hashing vectorizer.")
    op.add_option("--hack", default=False,
                  action="store_true", dest="hack",
                  help="use test instead on train to speedup process")
    op.add_option("--no-text", default=True,
                  action="store_false", dest="text",
                  help="features are text")
    op.add_option("--class_sample", default=2, type=int,
                  dest="n_sample_by_class",
                  help="show only [%default%] sample by class")
    op.add_option("--lnob", default=True, action='store_true',
                  dest='legend_outside_box',
                  help="legend not outside of the box")
    op.add_option("--legend", default=False, action='store_true',
                  dest='enable_legend_picking',
                  help='set legend picking not points')
    op.add_option("--noX", default=False, action='store_true',
                  dest='nox',
                  help="if you just want to generate graph and don't have acess to the X server ")
    op.add_option("-m","--methods", default=METHODS,
                  dest="methods",
                  help="dimension reduction method to try (split by ','); default = %s" %METHODS)
    op.add_option("-e", dest='exclude', default=None,
                  help="exclude class (separarated by ,)")
    op.add_option("-o", dest='only', default=None,
                  help="include only class (separarated by ,)")
    op.add_option("-v", dest='verbose', default=False, action='store_true',
                  help="verbose")

    (opts, args) = op.parse_args(args)

    if len(args) > 0:
        op.error("this script takes no arguments.")
        sys.exit(1)

    if opts.nox:
        matplotlib.use('Agg')
    # warning: pylab should be import after call to matplotlib.use(...)
    import pylab

    # load data
    print "loading {}".format(opts.fname)
    data_train, data_test = _load_map[opts.dataset](opts.fname)

    if opts.hack:
        print "hack: working on test dataset"
        data_train = data_test
        opts.dataset+='_test'

    if opts.verbose:
        print "----------example data loaded--------------"
        print "data:" ,data_train.data[0].strip()
        print "target:", data_train.target[0]
        print "-------------------------------------------"
    y_train, y_test = data_train.target, data_test.target

    data_train_size_mb = size_mb(data_train.data)
    data_test_size_mb = size_mb(data_test.data)

    print("%d documents - %0.3fMB (training set)" % (
            len(data_train.data), data_train_size_mb))
    print("%d documents - %0.3fMB (test set)" % (
            len(data_test.data), data_test_size_mb))


    print("Extracting features from the training dataset using a sparse vectorizer")
    t0 = time()
    if not opts.text:
        X_train = np.array(data_train.data, ndmin=2)
        features_names = data_train.features

    else: # its test dood
        if opts.use_hashing:
            print("Use feature hashing %s" %opts.n_features)
            vectorizer = HashingVectorizer(stop_words='english', non_negative=True,
                                           n_features=opts.n_features)
            X_train = vectorizer.transform(data_train.data)
        else:
            vectorizer = TfidfVectorizer(sublinear_tf=True, max_df=0.5,
                                         stop_words='english')
            # mapping from integer feature name to original token string
            X_train = vectorizer.fit_transform(data_train.data)
            feature_names = vectorizer.get_feature_names()

    if opts.verbose:
        print "----------example data transformed--------------"
        print "data:" ,X_train[0]
        print "target:", y_train[0]
        print "-------------------------------------------"

    duration = time() - t0
    print("done in %fs at %0.3fMB/s" % (duration, data_train_size_mb / duration))
    print("n_samples: %d, n_features: %d" % X_train.shape)
    print()

    print("Extracting features from the test dataset using the same vectorizer")
    t0 = time()
    if not opts.text:
        X_test = np.array(data_test.data, ndmin=2)
    else:
        X_test = vectorizer.transform(data_test.data)
    duration = time() - t0
    print("done in %fs at %0.3fMB/s" % (duration, data_test_size_mb / duration))
    print("n_samples: %d, n_features: %d" % X_test.shape)
    print()

    if opts.select_chi2!=-1:
        print("Extracting %d best features by a chi-squared test" %
              opts.select_chi2)
        t0 = time()
        ch2 = SelectKBest(chi2, k=opts.select_chi2)
        print "data:", X_train[0]
        print "target", y_train[0]
        X_train = ch2.fit_transform(X_train, y_train)
        X_test = ch2.transform(X_test)
        print("done in %fs" % (time() - t0))
        print()

    X = X_train.todense() if "todense" in dir(X_train) else X_train
    X_test = X_test.todense() if "todense" in dir(X_test) else X_test
    print "data shape: (%i,%i)" %(X.shape)

    if opts.only:
        idx = opts.only.split(',')
        X, y_train = filter_classes(X, y_train, idx, False)
        X_test, y_test = filter_classes(X_test, y_test, idx, False)

    if opts.exclude:
        idx = opts.exclude.split(',')
        X, y_train = filter_classes(X, y_train, idx, True)
        X_test, y_test = filter_classes(X_test, y_test, idx, True)


    # run all dim reduction algo
    for method in opts.methods.split(','):
        t0 = time()
        try:
            resdr = dim_reduce(method, X=X, Y=y_train)
            if resdr == None:
                continue
            trans,X_trans,title = resdr
            print('Projecting {} on test set'.format(method))
            if hasattr(trans,"transform"):
                X_trans_test = trans.transform(X_test)
            elif hasattr(trans,"fit_transform"):
                warnings.warn("the method as no transform (fallback to fit_transform", Warning)
                X_trans_test = trans.fit_transform(X_test)
            title = "%s (time %.2fs)" %(title,(time() - t0))
            print('Rendering plot {}'.format(title))
            has_plot = plot_embedding(X=X_trans_test, Y=y_test, title=title,
                              n_sample_by_class=opts.n_sample_by_class,
                              source=data_test.data,
                              legend_outside_box=opts.legend_outside_box,
                              enable_legend_picking=opts.enable_legend_picking)
            if has_plot:
                fname = "%s_%s.png" %(opts.dataset, method)
                print "saving %s" %fname
                pylab.savefig(fname, bbox_inches=0)
            else:
                print('Nothing to plot.')

        except Exception, ex:
            print method, ex
            print traceback.format_exc()

    pylab.show()

if __name__ == '__main__':
    main()

loading /app/.local/share/jupyter/runtime/kernel-fa9a7b8f-7390-4115-8748-69276c766de6.json
Loading 20 newsgroups dataset for categories:
['alt.atheism', 'talk.religion.misc', 'comp.graphics', 'sci.space']
data loaded
4 categories
()
2034 documents - 3.980MB (training set)
1353 documents - 2.867MB (test set)
Extracting features from the training dataset using a sparse vectorizer
done in 1.105438s at 3.600MB/s
n_samples: 2034, n_features: 33810
()
Extracting features from the test dataset using the same vectorizer
done in 0.653961s at 4.385MB/s
n_samples: 1353, n_features: 33810
()
data shape: (2034,33810)
Computing random projection
Projecting random on test set
Rendering plot Random Projection (time 3.95s)
saving news_random.png
Computing PCA projection

Traceback (most recent call last):
  File "<ipython-input-3-37edecb7a9d7>", line 153, in pca
    X_pca = pca.fit_transform(X)
  File "/app/.heroku/miniconda/lib/python2.7/site-packages/sklearn/decomposition/pca.py", line 653, in fit_transform
    X = self._fit(X)
  File "/app/.heroku/miniconda/lib/python2.7/site-packages/sklearn/decomposition/pca.py", line 602, in _fit
    full_var = np.var(X, axis=0).sum()
  File "/app/.heroku/miniconda/lib/python2.7/site-packages/numpy/core/fromnumeric.py", line 3089, in var
    keepdims=keepdims)



Computing LDA projection

  File "/app/.heroku/miniconda/lib/python2.7/site-packages/numpy/core/_methods.py", line 101, in _var
    x = asanyarray(arr - arrmean)
MemoryError
Traceback (most recent call last):



Computing Isomap embedding